In [1]:
import os

In [8]:
all_file = os.listdir(path="artifacts/04_25_2024_20_36_40/data_ingestion/features_store/archive (9)")

In [9]:
all_file

['classes.txt', 'coco128.yaml', 'train', 'val']

In [4]:
os.chdir("..")

In [6]:
os.system("unzip object_detection.zip")

1

In [8]:
import zipfile

with zipfile.ZipFile('object_detection.zip', 'r') as zip_ref:
    zip_ref.extractall()


In [4]:
%pwd

'd:\\Machine Learning project\\Object_detection\\Object-Detection\\research'

In [5]:
os.chdir("..")